In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import KFold



In [2]:
data_raw = np.load(r"c:\Users\wenze\Documents\02_Courses\3S_AW_24\ML\MLProject2024\fashion_train.npy")
data_img = [np.reshape(image[:784],(28,28)) for image in data_raw]

Below is the template matching with the averaged out templates for each category.
Results are surprisingly good, given that the model is pretty straightforward in the approach

In [49]:
def get_col_name(row):    
    b = (results_df.loc[row.name] == row['Template_score'])
    return b.index[b.argmax()]


kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(data_raw)
collect = []

for i,(train_index, test_index) in enumerate(kf.split(data_raw)):
    
    print(f"Fold {i}:")
    
    clothes = data_raw[train_index]
    test_clothes = data_raw[test_index]
    mean_clothes = [np.mean(clothes[clothes[:,-1]==x],axis=0) for x in range(5)]

    test_results = pd.DataFrame(test_index)
    results_df = pd.DataFrame()

    for i in range(len(mean_clothes)-1,-1,-1):
        results = np.sqrt(np.sum(np.square((test_clothes-mean_clothes[i])),axis=1))
        results_df.insert(0,i,pd.Series(results).values,allow_duplicates=True)
    
    test_results['Actual_score'] = test_clothes[:,-1]
    test_results.insert(2,"Template_score",results_df.min(axis = 1))
    test_results['Template_score'] = test_results.apply(get_col_name, axis=1)
                 
    y_test = test_results['Actual_score']
    y_pred = test_results['Template_score']
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
    print('Classification Report:\n',classification_report(y_test, y_pred))
    
    collect.append(confusion_matrix(y_test, y_pred))


Fold 0:
Confusion Matrix:
 [[296  18  13  54  22]
 [ 11 328   4  22   5]
 [  1   5 274   5 122]
 [ 31  25   1 326  40]
 [ 85  11 128  26 147]]
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.73      0.72       403
           1       0.85      0.89      0.87       370
           2       0.65      0.67      0.66       407
           3       0.75      0.77      0.76       423
           4       0.44      0.37      0.40       397

    accuracy                           0.69      2000
   macro avg       0.68      0.69      0.68      2000
weighted avg       0.68      0.69      0.68      2000

Fold 1:
Confusion Matrix:
 [[315  17   7  54  37]
 [  6 335   2  16  10]
 [  2   1 281   5 121]
 [ 19   7   2 334  23]
 [ 77   8 139  29 153]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.73      0.74       430
           1       0.91      0.91      0.91       369
           2  

Below is the template matching by taking the mean and dividing it over the variance, measuring the distance from that. Either I have done it in a not intended way or it is just not a reliable way of predicting values

In [48]:
def get_col_name(row):    
    b = (results_df.loc[row.name] == row['Template_score'])
    return b.index[b.argmax()]


kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(data_raw)
collect = []

for i,(train_index, test_index) in enumerate(kf.split(data_raw)):
    
    print(f"Fold {i}:")
    
    clothes = data_raw[train_index]
    test_clothes = data_raw[test_index]
    mean_clothes = [np.mean(clothes[clothes[:,-1]==x])/np.var(clothes[clothes[:,-1]==x]) for x in range(5)]

    test_results = pd.DataFrame(test_index)
    results_df = pd.DataFrame()

    for i in range(len(mean_clothes)-1,-1,-1):
        results = abs(np.mean(test_clothes,axis=1)/np.var(test_clothes,axis=1)-mean_clothes[i])
        results_df.insert(0,i,pd.Series(results).values,allow_duplicates=True)
    
    test_results['Actual_score'] = test_clothes[:,-1]
    test_results.insert(2,"Template_score",results_df.min(axis = 1))
    test_results['Template_score'] = test_results.apply(get_col_name, axis=1)
                 
    y_test = test_results['Actual_score']
    y_pred = test_results['Template_score']
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
    print('Classification Report:\n',classification_report(y_test, y_pred))
    
    collect.append(confusion_matrix(y_test, y_pred))
allones = np.array(collect).sum(axis=0)



Fold 0:
Confusion Matrix:
 [[ 94  10 232  37  28]
 [ 51 233  20 100   6]
 [ 69   1 280  15  20]
 [ 79 130  77  96  20]
 [ 46   4 309  13  30]]
Classification Report:
               precision    recall  f1-score   support

           0       0.28      0.23      0.25       401
           1       0.62      0.57      0.59       410
           2       0.31      0.73      0.43       385
           3       0.37      0.24      0.29       402
           4       0.29      0.07      0.12       402

    accuracy                           0.37      2000
   macro avg       0.37      0.37      0.34      2000
weighted avg       0.37      0.37      0.34      2000

Fold 1:
Confusion Matrix:
 [[ 88  13 227  37  44]
 [ 43 207  19  80   1]
 [ 63   0 323  21  20]
 [ 72 122  78 104  12]
 [ 33   2 350  22  19]]
Classification Report:
               precision    recall  f1-score   support

           0       0.29      0.22      0.25       409
           1       0.60      0.59      0.60       350
           2  